<a href="https://colab.research.google.com/github/Konayuma/HS/blob/main/hackingshield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random

def generate_sql_injections(num_samples=1000):
    injections = []
    usernames = ['admin', 'user', 'guest', 'test', 'root', 'alice', 'bob', 'charlie', 'eve', 'mallory']

    for _ in range(num_samples):
        username = random.choice(usernames)

        # List of SQL injection patterns
        injection_patterns = [
            f"SELECT * FROM users WHERE username = '{username}' --",
            f"SELECT * FROM users WHERE username = '{username}' OR '1'='1';",
            f"SELECT * FROM users WHERE username = '{username}' AND password = '' OR '1'='1';",
            f"SELECT * FROM users WHERE username = '{username}' UNION SELECT * FROM users;",
            f"SELECT * FROM users WHERE username = '{username}' AND 'x'='x';",
            f"SELECT * FROM users WHERE username = '{username}' OR EXISTS(SELECT * FROM users);",
            f"SELECT * FROM users WHERE username = '{username}' OR 1=1 LIMIT 1; --",
            f"SELECT * FROM users WHERE username = '{username}' AND 'a'='a' OR '1'='1';",
            f"SELECT * FROM users WHERE username = '{username}' OR (SELECT COUNT(*) FROM users) > 0;",
            f"SELECT * FROM users WHERE username = '{username}' AND (SELECT user FROM users WHERE '1'='1')='1';",
            f"SELECT * FROM users WHERE username = '{username}' AND password = 'password' OR 'x'='x';",
            f"SELECT * FROM users WHERE username = '{username}' AND (SELECT 1 FROM dual) IS NOT NULL;",
            f"SELECT * FROM users WHERE username = '{username}' UNION SELECT username, password FROM users;",
            f"SELECT * FROM users WHERE username = '{username}' AND '1'='2' UNION SELECT 'hacked', '123';"
        ]

        injection = random.choice(injection_patterns)
        injections.append(injection)

    return injections

# Generate the SQL injection samples
sql_injection_data = generate_sql_injections(1000)

# Save to a file
with open('sql_injection_samples.txt', 'w') as f:
    for line in sql_injection_data:
        f.write(line + '\n')

print("Generated 1000 SQL injection samples and saved to sql_injection_samples.txt.")


Generated 1000 SQL injection samples and saved to sql_injection_samples.txt.


In [2]:
import pandas as pd

# Load the SQL injection samples
with open('sql_injection_samples.txt', 'r') as file:
    injections = [(line.strip(), 1) for line in file.readlines()]  # Mark as harmful

# Load some benign queries as well (for demonstration)
benign_data = [
    ("SELECT * FROM users WHERE username = 'alice' AND password = 'securePass';", 0),
    ("SELECT username, email FROM users WHERE id = 1;", 0),
    ("SELECT COUNT(*) FROM users;", 0),
]

# Combine the datasets
data = injections + benign_data
df = pd.DataFrame(data, columns=['query', 'label'])


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF vectors
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['query'])
y = df['label']


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[201]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       201

    accuracy                           1.00       201
   macro avg       1.00      1.00      1.00       201
weighted avg       1.00      1.00      1.00       201



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [5]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 100.00%


In [6]:
def predict_injection(query):
    query_vector = vectorizer.transform([query])
    prediction = model.predict(query_vector)
    return "Harmful" if prediction[0] == 1 else "Benign"

# Example usage
print(predict_injection("SELECT * FROM users WHERE username = 'admin' --"))
print(predict_injection("SELECT * FROM users WHERE username = 'alice' AND password = 'securePass';"))


Harmful
Harmful


In [7]:
# List of harmless SQL queries to test
harmless_queries = [
    "SELECT * FROM users WHERE username = 'alice' AND password = 'securePass';",
    "SELECT username, email FROM users WHERE id = 1;",
    "SELECT COUNT(*) FROM users;",
    "SELECT * FROM products WHERE price < 100;",
    "SELECT name FROM employees WHERE department = 'sales';"
]

# Test the model with harmless queries
for query in harmless_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")


Query: SELECT * FROM users WHERE username = 'alice' AND password = 'securePass'; -> Prediction: Harmful
Query: SELECT username, email FROM users WHERE id = 1; -> Prediction: Harmful
Query: SELECT COUNT(*) FROM users; -> Prediction: Harmful
Query: SELECT * FROM products WHERE price < 100; -> Prediction: Harmful
Query: SELECT name FROM employees WHERE department = 'sales'; -> Prediction: Harmful


The IMmroved version

In [8]:
import random
import pandas as pd

# Function to generate harmful SQL injection queries
def generate_harmful_queries(num_samples=1000):
    usernames = ['admin', 'user', 'guest', 'test', 'root', 'alice', 'bob', 'charlie', 'eve', 'mallory']
    injections = []

    for _ in range(num_samples):
        username = random.choice(usernames)
        injection_patterns = [
            f"SELECT * FROM users WHERE username = '{username}' --",
            f"SELECT * FROM users WHERE username = '{username}' OR '1'='1';",
            f"SELECT * FROM users WHERE username = '{username}' AND password = '' OR '1'='1';",
            f"SELECT * FROM users WHERE username = '{username}' UNION SELECT * FROM users;",
            f"SELECT * FROM users WHERE username = '{username}' AND 'x'='x';",
            f"SELECT * FROM users WHERE username = '{username}' OR EXISTS(SELECT * FROM users);",
            f"SELECT * FROM users WHERE username = '{username}' OR 1=1 LIMIT 1; --",
            f"SELECT * FROM users WHERE username = '{username}' AND '1'='2' UNION SELECT 'hacked', '123';"
        ]
        injection = random.choice(injection_patterns)
        injections.append((injection, 1))  # Mark as harmful (1)

    return injections

# Function to generate harmless SQL queries
def generate_harmless_queries(num_samples=1000):
    benign_queries = [
        "SELECT * FROM users WHERE username = 'alice' AND password = 'securePass';",
        "SELECT username, email FROM users WHERE id = 1;",
        "SELECT COUNT(*) FROM users;",
        "SELECT * FROM products WHERE price < 100;",
        "SELECT name FROM employees WHERE department = 'sales';",
        "INSERT INTO users (username, password) VALUES ('testUser', 'testPass');",
        "UPDATE users SET last_login = NOW() WHERE username = 'bob';",
        "DELETE FROM users WHERE id = 10;"
    ]

    harmless_samples = []
    for _ in range(num_samples):
        harmless_query = random.choice(benign_queries)
        harmless_samples.append((harmless_query, 0))  # Mark as harmless (0)

    return harmless_samples

# Generate both harmful and harmless queries
harmful_data = generate_harmful_queries(1000)
harmless_data = generate_harmless_queries(1000)

# Combine data and create DataFrame
data = harmful_data + harmless_data
df = pd.DataFrame(data, columns=['query', 'label'])

# Shuffle the DataFrame to mix harmful and harmless queries
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save to a CSV file
df.to_csv('sql_queries_dataset.csv', index=False)

print("Generated 2000 SQL queries (1000 harmful and 1000 harmless) and saved to sql_queries_dataset.csv.")


Generated 2000 SQL queries (1000 harmful and 1000 harmless) and saved to sql_queries_dataset.csv.


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Load the dataset
df = pd.read_csv('sql_queries_dataset.csv')

# Step 2: Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['query'])
y = df['label']

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


[[203   0]
 [  0 197]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       203
           1       1.00      1.00      1.00       197

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400

Model Accuracy: 100.00%


In [10]:
# List of harmful SQL queries to test
harmful_queries = [
    "SELECT * FROM users WHERE username = 'admin' --",
    "SELECT * FROM users WHERE username = 'user' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'guest' AND password = '' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'admin' UNION SELECT * FROM users;",
    "SELECT * FROM users WHERE username = 'bob' AND '1'='1';"
]

# List of harmless SQL queries to test
harmless_queries = [
    "SELECT * FROM users WHERE username = 'alice' AND password = 'securePass';",
    "SELECT username, email FROM users WHERE id = 1;",
    "SELECT COUNT(*) FROM users;",
    "SELECT * FROM products WHERE price < 100;",
    "INSERT INTO users (username, password) VALUES ('newUser', 'newPass');"
]

# Function to predict injection type
def predict_injection(query):
    query_vector = vectorizer.transform([query])
    prediction = model.predict(query_vector)
    return "Harmful" if prediction[0] == 1 else "Benign"

# Test the model with harmful queries
print("Testing Harmful Queries:")
for query in harmful_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")

# Test the model with harmless queries
print("\nTesting Harmless Queries:")
for query in harmless_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")


Testing Harmful Queries:
Query: SELECT * FROM users WHERE username = 'admin' -- -> Prediction: Harmful
Query: SELECT * FROM users WHERE username = 'user' OR '1'='1'; -> Prediction: Harmful
Query: SELECT * FROM users WHERE username = 'guest' AND password = '' OR '1'='1'; -> Prediction: Harmful
Query: SELECT * FROM users WHERE username = 'admin' UNION SELECT * FROM users; -> Prediction: Harmful
Query: SELECT * FROM users WHERE username = 'bob' AND '1'='1'; -> Prediction: Harmful

Testing Harmless Queries:
Query: SELECT * FROM users WHERE username = 'alice' AND password = 'securePass'; -> Prediction: Benign
Query: SELECT username, email FROM users WHERE id = 1; -> Prediction: Benign
Query: SELECT COUNT(*) FROM users; -> Prediction: Benign
Query: SELECT * FROM products WHERE price < 100; -> Prediction: Benign
Query: INSERT INTO users (username, password) VALUES ('newUser', 'newPass'); -> Prediction: Benign


Adding llms to report harmful behavoirs

In [11]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [12]:
from groq import Groq

api_key= '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'
# Function to send an alert email
def send_alert_email(harmful_query):
    client = Groq(api_key = api_key)
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "system",
                "content": "This is an LLM for reporting SQL injections to the user from the security team. Create an email for the alert."
            },
            {
                "role": "user",
                "content": f"A harmful SQL injection has been detected: {harmful_query}"
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    email_content = ""
    for chunk in completion:
        email_content += chunk.choices[0].delta.content or ""

    print("Sending email alert...")
    print(email_content)

# Modify the prediction testing code
for query in harmful_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")
    if result == "Harmful":
        send_alert_email(query)


Query: SELECT * FROM users WHERE username = 'admin' -- -> Prediction: Harmful
Sending email alert...
Subject: SQL Injection Alert - User Identification

Dear [User],

We have detected a potential security vulnerability on our system and wanted to bring it to your attention. A SQL injection attack has been identified, which could potentially compromise the integrity of our database. The malicious SQL query has been detected as:

SELECT * FROM users WHERE username = '/admin' --

This unauthorized query has been attempting to gain access to our system with a username of 'admin'. We take data security very seriously and want to ensure that our system remains secure.

Here is the action plan we suggest:

1. We recommend that you do not click on any suspicious links or respond to any unsolicited requests.
2. Do not attempt to log in to your account or access any sensitive information until we have further instructions.
3. We are working on remediating this issue promptly and will notify you 

In [13]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_alert_email(harmful_query):
    sender_email = "sepokonayuma@gmail.com"  # Your Gmail address
    receiver_email = "plastalbotbuilders@gmail.com"  # Recipient's email address
    password = "ahjzhgazffxaxhyg"  # Your Gmail password or App Password

    # Create the email content
    subject = "SQL Injection Alert"
    body = f"""
    Dear Sir/Madam,

    A harmful SQL injection has been detected:

    Query: {harmful_query}

    Please take necessary action.

    Best regards,
    Your Security System
    """

    # Set up the email
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Send the email
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Upgrade the connection to secure
            server.login(sender_email, password)  # Log in to your Gmail account
            server.send_message(msg)  # Send the email
        print("Email alert sent successfully.")
    except Exception as e:
        print(f"Error sending email: {e}")


In [14]:
# Function to predict injection type
def predict_injection(query):
    query_vector = vectorizer.transform([query])
    prediction = model.predict(query_vector)
    return "Harmful" if prediction[0] == 1 else "Benign"

In [15]:
# Define harmful queries
harmful_queries = [
    "SELECT * FROM users WHERE username = 'admin' --",
    "SELECT * FROM users WHERE username = 'user' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'guest' AND password = '' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'admin' UNION SELECT * FROM users;",
    "SELECT * FROM users WHERE username = 'bob' AND '1'='1';"
]

# Test the model with harmful queries
for query in harmful_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")
    if result == "Harmful":
        send_alert_email(query)

Query: SELECT * FROM users WHERE username = 'admin' -- -> Prediction: Harmful
Email alert sent successfully.
Query: SELECT * FROM users WHERE username = 'user' OR '1'='1'; -> Prediction: Harmful
Email alert sent successfully.
Query: SELECT * FROM users WHERE username = 'guest' AND password = '' OR '1'='1'; -> Prediction: Harmful
Email alert sent successfully.
Query: SELECT * FROM users WHERE username = 'admin' UNION SELECT * FROM users; -> Prediction: Harmful
Email alert sent successfully.
Query: SELECT * FROM users WHERE username = 'bob' AND '1'='1'; -> Prediction: Harmful
Email alert sent successfully.


In [16]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Load the dataset
try:
    df = pd.read_csv('sql_queries_dataset.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: Dataset not found. Please ensure the dataset is in the correct path.")
    raise

# Step 2: Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['query'])
y = df['label']
print("TF-IDF feature extraction completed.")

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the model
model = LogisticRegression()
model.fit(X_train, y_train)
print("Model training completed.")

# Step 5: Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Function to predict injection type
def predict_injection(query):
    query_vector = vectorizer.transform([query])
    prediction = model.predict(query_vector)
    return "Harmful" if prediction[0] == 1 else "Benign"

# Function to send an email alert with enhanced logging and error handling
def send_alert_email(harmful_query):
    sender_email = "sepokonayuma@gmail.com"  # Your Gmail address
    receiver_email = "plastalbotbuilders@gmail.com"  # Recipient's email address
    password = "ahjzhgazffxaxhyg"  # Your Gmail password or App Password

    # Create the email content with more informative alert
    subject = "🚨 SQL Injection Alert - Harmful Query Detected"
    body = f"""
    Dear Security Team,

    An SQL injection attempt has been detected and flagged as harmful by the security system.

    ⚠️ Harmful Query: {harmful_query}

    Please review the query and take immediate action to prevent any security breaches.

    Best regards,
    Automated SQL Injection Detection System
    """

    # Set up the email
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Send the email
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Upgrade the connection to secure
            server.login(sender_email, password)  # Log in to your Gmail account
            server.send_message(msg)  # Send the email
        print(f"🚀 Alert email sent successfully to {receiver_email}.")
    except Exception as e:
        print(f"❌ Error sending email: {e}")

# Step 6: Define harmful queries (improved with more realistic SQL injections)
harmful_queries = [
    "SELECT * FROM users WHERE username = 'admin' --",
    "SELECT * FROM users WHERE username = 'user' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'guest' AND password = '' OR '1'='1';",
    "SELECT * FROM users WHERE username = 'admin' UNION SELECT * FROM users;",
    "SELECT * FROM users WHERE username = 'bob' AND '1'='1';",
    "DROP TABLE users; --",
    "SELECT * FROM users WHERE password = 'abc' AND '1'='1';",
]

# Step 7: Test the model with harmful queries and log the results
print("Starting prediction and alert process for harmful queries...")

for query in harmful_queries:
    result = predict_injection(query)
    print(f"Query: {query} -> Prediction: {result}")
    if result == "Harmful":
        send_alert_email(query)

print("Prediction and alert process completed.")


Dataset loaded successfully.
TF-IDF feature extraction completed.
Model training completed.
Model Accuracy: 100.00%
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       203
           1       1.00      1.00      1.00       197

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400

Confusion Matrix:
 [[203   0]
 [  0 197]]
Starting prediction and alert process for harmful queries...
Query: SELECT * FROM users WHERE username = 'admin' -- -> Prediction: Harmful
🚀 Alert email sent successfully to plastalbotbuilders@gmail.com.
Query: SELECT * FROM users WHERE username = 'user' OR '1'='1'; -> Prediction: Harmful
🚀 Alert email sent successfully to plastalbotbuilders@gmail.com.
Query: SELECT * FROM users WHERE username = 'guest' AND password = '' OR '1'='1'; -> Prediction: Harmful
🚀 Alert email sent successfully

In [17]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


In [18]:
!wget -q -O - ipv4.icanhazip.com

34.106.84.17


In [25]:
%%writefile app.py
import streamlit as st
import pandas as pd
import smtplib
import os
import logging
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

# Ensure the log file exists
log_file = 'app.log'
if not os.path.exists(log_file):
    open(log_file, 'w').close()

# Configure logging
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Step 1: Load the dataset
df = pd.read_csv('sql_queries_dataset.csv')

# Step 2: Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['query'])
y = df['label']

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Function to predict injection type
def predict_injection(query):
    query_vector = vectorizer.transform([query])
    prediction = model.predict(query_vector)
    return "Harmful" if prediction[0] == 1 else "Benign"

# Function to send an email alert
def send_alert_email(harmful_query):
    sender_email = "sepokonayuma@gmail.com"  # Your Gmail address
    receiver_email = "plastalbotbuilders@gmail.com"  # Recipient's email address
    password = "your_password_here"  # Your Gmail password or App Password

    # Create the email content
    subject = "SQL Injection Alert"
    body = f"""
    Dear Sir/Madam,

    A harmful SQL injection has been detected:

    Query: {harmful_query}

    Please take necessary action.

    Best regards,
    Your Security System
    """

    # Set up the email
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Send the email
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Upgrade the connection to secure
            server.login(sender_email, password)  # Log in to your Gmail account
            server.send_message(msg)  # Send the email
        logger.info("Email alert sent successfully.")
    except Exception as e:
        logger.error(f"Error sending email: {e}")

# Streamlit UI with Sidebar
st.sidebar.title("Navigation")
options = st.sidebar.radio("Select an option:", ["User Dashboard", "Predict SQL Query", "Query Patterns Visualization", "Logs"])

# Bento Box Design
def bento_box(label, description, option_name):
    """Create a clickable bento box that redirects to another page."""
    if st.button(label):
        st.session_state.page = option_name  # Navigate to the correct page
    st.write(f"**{description}**")
    st.markdown("---")

# Dashboard Default Page
if 'page' not in st.session_state:
    st.session_state.page = 'User Dashboard'

if st.session_state.page == "User Dashboard":
    st.title("User Dashboard")

    # Bento Box for Current Status
    st.subheader("Current Status")
    bento_box("View Current Status", "View current system activity, including query trends and predictions.", "Query Patterns Visualization")

    # Bento Box for SQL Query Prediction
    st.subheader("SQL Injection Detection")
    bento_box("Predict SQL Query", "Enter a new SQL query to test for potential injection vulnerabilities.", "Predict SQL Query")

    # Bento Box for Logs
    st.subheader("Log Summary")
    # Display a summary of the logs
    with open(log_file, "r") as f:
        logs = f.readlines()[-5:]  # Show the last 5 log entries
    st.text_area("Recent Logs", value=''.join(logs), height=150)

    bento_box("View Full Logs", "Explore system logs for detailed query and prediction history.", "Logs")

    # Bento Box for Visualization
    st.subheader("Query Patterns")
    bento_box("View Query Patterns", "View a visualization of harmful and benign queries on a Cartesian plane.", "Query Patterns Visualization")

    # Navigation Buttons at the bottom
    st.markdown("---")
    if st.button("Go to SQL Injection Prediction"):
        st.session_state.page = "Predict SQL Query"
    if st.button("Go to Query Visualization"):
        st.session_state.page = "Query Patterns Visualization"
    if st.button("View Logs"):
        st.session_state.page = "Logs"

# Predict SQL Query Page
if st.session_state.page == "Predict SQL Query":
    st.title("SQL Injection Detection System")

    # Input for SQL query
    sql_query = st.text_input("Enter SQL Query:")

    if st.button("Predict"):
        if sql_query:
            result = predict_injection(sql_query)
            st.write(f"**Prediction:** {result}")

            # Log the query and prediction
            logger.info(f"Query: {sql_query} -> Prediction: {result}")

            # Send email if harmful
            if result == "Harmful":
                send_alert_email(sql_query)
                st.warning("Alert email sent for harmful query.")

    # Navigation Buttons at the bottom
    if st.button("Back to Dashboard"):
        st.session_state.page = "User Dashboard"
    if st.button("View Query Patterns"):
        st.session_state.page = "Query Patterns Visualization"
    if st.button("View Logs"):
        st.session_state.page = "Logs"

# Query Patterns Visualization Page
if st.session_state.page == "Query Patterns Visualization":
    st.subheader("Query Patterns Visualization on Cartesian Plane")

    # Prepare data for visualization
    df['label'] = df['label'].map({0: 'Benign', 1: 'Harmful'})  # Map labels for visualization

    # Create a scatter plot
    fig, ax = plt.subplots()

    # Scatter plot for harmful queries
    harmful_queries = df[df['label'] == 'Harmful']
    benign_queries = df[df['label'] == 'Benign']

    ax.scatter(harmful_queries.index, [1] * len(harmful_queries), color='red', label='Harmful', alpha=0.6)
    ax.scatter(benign_queries.index, [0] * len(benign_queries), color='green', label='Benign', alpha=0.6)

    ax.set_yticks([0, 1])
    ax.set_yticklabels(['Benign', 'Harmful'])
    ax.set_xlabel('Queries')
    ax.set_title('Harmful vs Benign Queries on Cartesian Plane')
    ax.legend()

    st.pyplot(fig)

    # Navigation Buttons at the bottom
    if st.button("Back to Dashboard"):
        st.session_state.page = "User Dashboard"
    if st.button("Go to SQL Injection Prediction"):
        st.session_state.page = "Predict SQL Query"
    if st.button("View Logs"):
        st.session_state.page = "Logs"

# Logs Page
if st.session_state.page == "Logs":
    st.subheader("Logs")
    log_output = st.empty()  # Placeholder for log output

    # Function to fetch and display logs
    def display_logs():
        with open(log_file, "r") as f:
            log_content = f.read()
        log_output.text_area("Log Output", value=log_content, height=300)

    # Display logs when requested
    if st.button("Show Logs"):
        display_logs()

    # Navigation Buttons at the bottom
    if st.button("Back to Dashboard"):
        st.session_state.page = "User Dashboard"
    if st.button("Go to SQL Injection Prediction"):
        st.session_state.page = "Predict SQL Query"
    if st.button("View Query Patterns"):
        st.session_state.page = "Query Patterns Visualization"


Overwriting app.py


In [ ]:
!npm install -g npx


npm ERR! code EEXIST
npm ERR! path /tools/node/bin/npx
npm ERR! EEXIST: file already exists
npm ERR! File exists: /tools/node/bin/npx
npm ERR! Remove the existing file and try again, or run npm
npm ERR! with --force to overwrite files recklessly.

npm ERR! A complete log of this run can be found in:
npm ERR!     /root/.npm/_logs/2024-10-05T09_50_23_598Z-debug-0.log


In [26]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.84.17:8501

your url is: https://dirty-points-flow.loca.lt
2024-10-07 07:36:42.221 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/exec_code.py", line 88, in exec_func_with_error_handling
    result = func()
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 579, in code_to_exec
    exec(code, module.__dict__)
  File "/content/app.py", line 186, in <module>
    if st.button("Go to SQL Injection Prediction"):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/metrics_util.py", line 410, in wrapped_func
    result = non_optional_func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/streamlit/elements/widgets/button.py", line 222, in button
    return self.dg.

In [ ]:
!pip install flask-ngrok
!ngrok authtoken '2n11GDYgmPJnIEjGM05p2sbN796_ADxyzVWPVoBr2L1y6Mtz'

/bin/bash: line 1: ngrok: command not found


In [27]:
!pip install flask pyngrok


In [33]:
from flask import Flask, render_template, request, redirect, url_for, flash
import pandas as pd
import smtplib
import os
import logging
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

# Initialize Flask app
app = Flask(__name__)
app.secret_key = os.urandom(24)

# Ensure the log file exists
log_file = 'app.log'
if not os.path.exists(log_file):
    open(log_file, 'w').close()

# Configure logging
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Step 1: Load the dataset
df = pd.read_csv('sql_queries_dataset.csv')

# Step 2: Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['query'])
y = df['label']

# Step 3: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Function to predict injection type
def predict_injection(query):
    query_vector = vectorizer.transform([query])
    prediction = model.predict(query_vector)
    return "Harmful" if prediction[0] == 1 else "Benign"

# Function to send an email alert
def send_alert_email(harmful_query):
    sender_email = "sepokonayuma@gmail.com"  # Your Gmail address
    receiver_email = "plastalbotbuilders@gmail.com"  # Recipient's email address
    password = "zatkvybrqqjmauow"  # Your Gmail password or App Password

    # Create the email content
    subject = "SQL Injection Alert"
    body = f"""
    Dear Sir/Madam,

    A harmful SQL injection has been detected:

    Query: {harmful_query}

    Please take necessary action.

    Best regards,
    Your Security System
    """

    # Set up the email
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Send the email
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Upgrade the connection to secure
            server.login(sender_email, password)  # Log in to your Gmail account
            server.send_message(msg)  # Send the email
        logger.info("Email alert sent successfully.")
    except Exception as e:
        logger.error(f"Error sending email: {e}")

# Flask routes

# Route for the homepage
@app.route('/')
def home():
    return render_template('index.html')

# Route for SQL query prediction
@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        sql_query = request.form['query']
        if sql_query:
            result = predict_injection(sql_query)

            # Log the query and prediction
            logger.info(f"Query: {sql_query} -> Prediction: {result}")

            # Send email if harmful
            if result == "Harmful":
                send_alert_email(sql_query)
                flash("Alert email sent for harmful query.", "warning")

            return render_template('predict.html', result=result)

    return render_template('predict.html')

# Route for query patterns visualization
@app.route('/visualize')
def visualize():
    df['label'] = df['label'].map({0: 'Benign', 1: 'Harmful'})  # Map labels for visualization

    # Create a scatter plot
    fig, ax = plt.subplots()
    harmful_queries = df[df['label'] == 'Harmful']
    benign_queries = df[df['label'] == 'Benign']

    ax.scatter(harmful_queries.index, [1] * len(harmful_queries), color='red', label='Harmful', alpha=0.6)
    ax.scatter(benign_queries.index, [0] * len(benign_queries), color='green', label='Benign', alpha=0.6)

    ax.set_yticks([0, 1])
    ax.set_yticklabels(['Benign', 'Harmful'])
    ax.set_xlabel('Queries')
    ax.set_title('Harmful vs Benign Queries on Cartesian Plane')
    ax.legend()

    # Save the plot as an image and display it
    img_path = "static/visualization.png"
    plt.savefig(img_path)
    return render_template('visualize.html', img_path=img_path)

# Route for viewing logs
@app.route('/logs')
def logs():
    with open(log_file, "r") as f:
        log_content = f.readlines()[-20:]  # Display the last 20 logs
    return render_template('logs.html', logs=log_content)

# Run Flask app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [32]:
import os
secret_key = os.urandom(24)
print(secret_key)


b'\xe7\xaa\xe9qy\x90ze\xfe\x8ev\xe7\xcaT\xe8;\xab\x00\x96\xfc\xae\xcdp\xbc'


In [ ]:
from pyngrok import ngrok

# Terminate any existing tunnels (if any)
ngrok.kill()

# Set your authtoken (replace with your actual token)
ngrok.set_auth_token("2n11GDYgmPJnIEjGM05p2sbN796_ADxyzVWPVoBr2L1y6Mtz")

# Expose the Flask app on the internet
public_url = ngrok.connect(5000)
print("Your Flask app is running here:", public_url)

# Start the Flask app
app.run(port=5000)

Your Flask app is running here: NgrokTunnel: "https://5f25-34-106-84-17.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [38]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok


In [40]:
app.run()


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [35]:
import os
app.secret_key = os.urandom(24)  # Generate a random secret key


In [ ]:
from google.colab import files

# Download the CSV file
files.download('sql_queries_dataset.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>